# 🔮 QEPC Daily Predictions

**Quantum Enhanced Prediction Calculator v2.0**

Features:
- 🎲 Quantum-inspired Monte Carlo simulation
- 📊 Recency-weighted team strengths
- 💰 **Vegas odds comparison** - Find value bets!
- ⭐ **Edge detection** - Where we disagree with Vegas

---

In [ ]:
# SETUP - Run this first!
import sys
from pathlib import Path

# Your paths
project_root = Path(r"C:\Users\wdors\qepc_project")
qepc_v2_path = project_root / "experimental" / "CLAUDE_REWRITE" / "qepc_v2"

if str(qepc_v2_path) not in sys.path:
    sys.path.insert(0, str(qepc_v2_path))

print(f"📁 Project root: {project_root}")
print(f"📦 QEPC v2 code: {qepc_v2_path}")

# Import QEPC
from qepc.data.loader import DataLoader
from qepc.sports.nba.predictor import GamePredictor, find_edges

print(f"\n✅ QEPC v2.0 loaded!")

---
## 📊 Initialize & Load Data

In [ ]:
# Initialize with your project root
loader = DataLoader(project_root=project_root)
predictor = GamePredictor(data_loader=loader)

# Prepare (loads all data including Vegas odds)
predictor.prepare(verbose=True)

---
## 🏆 Power Rankings

In [ ]:
rankings = predictor.get_power_rankings()
print("\n🏆 POWER RANKINGS")
print("=" * 70)
rankings.head(15)

---
## 🏀 Today's Predictions

In [ ]:
print("\n🔮 TODAY'S PREDICTIONS")
print("=" * 70)

predictions = predictor.predict_today(verbose=True)

if predictions:
    print(f"\n✅ Generated {len(predictions)} predictions")
else:
    print("\n⚠️  No games found for today")

In [ ]:
# Show predictions as table
if predictions:
    df = predictor.predictions_to_dataframe(predictions)
    display(df)

---
## ⭐ Find Edges (QEPC vs Vegas)

Games where QEPC disagrees with Vegas by **2+ points** = potential value!

In [ ]:
# Find value bets
edges = predictor.find_edges(predictions)

if not edges:
    print("\nNo significant edges found today.")
    print("QEPC and Vegas are mostly in agreement.")

---
## 🎯 Predict a Specific Game

In [ ]:
# Change these team names!
HOME_TEAM = "Cleveland Cavaliers"
AWAY_TEAM = "Boston Celtics"

pred = predictor.predict_game(HOME_TEAM, AWAY_TEAM)

if pred:
    print(f"\n🏀 {AWAY_TEAM} @ {HOME_TEAM}")
    print("=" * 50)
    print(f"""
🎯 PREDICTION
   Winner: {pred.predicted_winner}
   Win Probability: {max(pred.home_win_prob, pred.away_win_prob):.1%}
   
📊 SCORE
   {AWAY_TEAM}: {pred.away_score:.0f}
   {HOME_TEAM}: {pred.home_score:.0f}
   
📏 QEPC LINE
   Spread: {HOME_TEAM} {pred.predicted_spread:+.1f}
   Total: {pred.predicted_total:.0f}
    """)
    
    if pred.vegas and pred.vegas.vegas_spread is not None:
        v = pred.vegas
        print(f"""💰 VEGAS COMPARISON
   Vegas Spread: {v.vegas_spread:+.1f}
   QEPC Spread: {v.qepc_spread:+.1f}
   Difference: {v.spread_diff:+.1f} pts
        """)
        if pred.has_vegas_edge:
            print(f"   ⭐ EDGE: Bet {v.edge} ({v.edge_size:.1f} pt edge)")
    else:
        print("   (Vegas odds not available for this game)")
else:
    print(f"❌ Could not find teams")
    print(f"\n💡 Available teams:")
    for team in sorted(predictor._strengths.keys())[:10]:
        print(f"   {team}")

---
## 💾 Save Predictions

In [ ]:
if predictions:
    from datetime import datetime
    
    output_dir = project_root / 'data' / 'results' / 'predictions'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    today = datetime.now().strftime('%Y%m%d_%H%M')
    filename = f"QEPC_Predictions_{today}.csv"
    output_path = output_dir / filename
    
    df = predictor.predictions_to_dataframe(predictions)
    df.to_csv(output_path, index=False)
    
    print(f"💾 Saved to: {output_path}")

---

## 📚 Understanding the Output

| Column | Meaning |
|--------|--------|
| `QEPC_Spread` | Our predicted spread (negative = away team favored) |
| `Vegas_Spread` | The Vegas line |
| `Spread_Diff` | QEPC - Vegas. Positive = we like home more |
| `Edge` | HOME or AWAY - which side has value |
| `Edge_Size` | How many points we disagree by |

### What is an "Edge"?

An **edge** is when QEPC disagrees with Vegas by **2+ points**.

- If `Spread_Diff > 2`: We like the HOME team more than Vegas → Bet HOME
- If `Spread_Diff < -2`: We like the AWAY team more than Vegas → Bet AWAY

⚠️ **Disclaimer**: This is for entertainment/research only. Always gamble responsibly.